In [ ]:
from collections import Counter
import sys

In [ ]:
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import sklearn.cluster
import sklearn.decomposition
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import xarray as xr

In [ ]:
sys.path.append('../src/lib/')
import plot

## Load Data ##

Factors calculated on the Imagene gene expression data set.

In [ ]:
sfa = xr.open_dataset('../models/sfa.nc')
sfa['factor_name'] = ('factor', np.array([
    'ER',
    'EMT',
    'Luminal Proliferative',
    'Technical RNA-seq',
    'Technical RPPA',
    'Immune',
    'HER2',
    'Normal-like / ILC',
    'Basal',
    'Chr8'
], 'object'))
sfa

MRI features from the same patients.

In [ ]:
mri = xr.open_dataset('../data/processed/mri-features.nc')
mri = mri.reindex(patient=sfa['patient'])
assert all(mri['patient'].values == sfa['patient'].values)
mri

And clinical variables.

In [ ]:
clin = pd.read_table('../data/raw/imagene_clinical.tsv')
clin = clin.set_index('margins_patient')
clin.index.name = 'patient'
clin = clin.to_xarray()
clin = clin.reindex(patient=sfa['patient'])
assert(np.all(clin['patient'].values == sfa['patient'].values))
clin

In [ ]:
Counter(clin['ihc_subtype'].values)

Select samples with no missing values for MRI features, and put them into an array.

In [ ]:
mri_features = list(set(mri.keys()) - {'patient', 'Comment', 'MultiFocal'})
mri_array = mri[mri_features].to_array()

## Supporting Functions ##

In [ ]:
def plot_roc(y_true, y_score):
    """Plot a ROC curve for binary true y and predicted y scores."""
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true, y_score)
    auc = sklearn.metrics.auc(fpr, tpr)
    with plot.subplots(1, 1) as (fig, ax):
        ax.plot(fpr, tpr)
        ax.plot([0, 1], [0, 1], color='black', linestyle='--')
        ax.set_title("AUC = {}".format(auc))

In [ ]:
def bin_performance(y, y_pred):
    """Display performance for binary classification."""
    if y_pred.dtype != np.bool:
        y_pred = y_pred > 0.5
    
    return dict({
        'Accuracy': np.mean(y == y_pred),
        'Sensitivity': np.sum(y & y_pred) / np.sum(y),
        'Specificity': np.sum(~y & ~y_pred) / np.sum(~y),
        'Precision': np.sum(y & y_pred) / np.sum(y_pred)
    })

In [ ]:
def mse_performance(y, y_pred):
    """"Display performance by mean square error."""
    if y_pred.dtype != np.bool:
        y_pred = y_pred > 0.5
    
    return dict({
        'MSE': np.mean((y - y_pred)**2)
    })

In [ ]:
class PcaLog():
    """Predictive model by logistic regression of principal components."""
    
    def __init__(self, n_components):
        self.n_components = n_components

    def train(self, X, y):

        self._pca = sklearn.decomposition.PCA(self.n_components)
        X_pc = self._pca.fit_transform(X)

        self._logm = sklearn.linear_model.LogisticRegression()
        self._logm.fit(X_pc, y)
        
    def predict(self, X):
        X_pc = self._pca.transform(X)
        return self._logm.predict_proba(X_pc)[:, 1]

In [ ]:
class PcaLin():
    """Predictive model by linear regression of principal components."""
    
    def __init__(self, n_components):
        self.n_components = n_components

    def train(self, X, y):

        self._pca = sklearn.decomposition.PCA(self.n_components)
        X_pc = self._pca.fit_transform(X)

        self._lm = sklearn.linear_model.LinearRegression()
        self._lm.fit(X_pc, y)
        
    def predict(self, X):
        X_pc = self._pca.transform(X)
        return self._lm.predict(X_pc)

# Predict ER status from MRI features #

This as a sanity check for the method. We know we should be able to predict ER status to some extent, and predicting the ER factor should perform similarly.

In [ ]:
sel_patients = list(
    {p for t, p in zip(clin['ihc_subtype'], clin['patient'].values)
     if t in ['ER+/HER2-', 'TN']} &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values
y = clin['ihc_subtype'].reindex(patient=sel_patients).values == 'ER+/HER2-'

display(Markdown("Number of samples: {}".format(X.shape[0])))

display(Markdown("Performance under null model:"))
display(bin_performance(y, np.ones(len(y))))

display(Markdown("**Logistic PC Regresion**"))

model = PcaLog(10)
model.train(X, y)
y_pred = model.predict(X)
display(bin_performance(y, y_pred))
plot_roc(y, y_pred)

display(Markdown("**LOOCV Logistic PC Regresion**"))

loocv = sklearn.model_selection.LeaveOneOut()
y_pred = np.full(y.shape, np.nan)
for train_index, test_index in loocv.split(X):
    m = PcaLog(10)
    m.train(X[train_index, :], y[train_index])
    y_pred[test_index] = m.predict(X[test_index, :])
display(bin_performance(y, y_pred))
plot_roc(y, y_pred)

# Predict Factors from MRI features in all subtypes #

## Linear Regression ##

In [ ]:
sel_patients = list(
    set(sfa['patient'].values) &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values
y = sfa['factors'].reindex(patient=sel_patients).transpose('patient', 'factor').values
y = (y - np.mean(y, 0, keepdims=True)) / np.std(y, 0, keepdims=True)

display(Markdown("Number of samples: {}".format(X.shape[0])))

for factor_i in range(y.shape[1]):
    loocv = sklearn.model_selection.LeaveOneOut()
    y_pred = np.full(y.shape[0], np.nan)
    for train_index, test_index in loocv.split(X):
        m = PcaLin(10)
        m.train(X[train_index, :], y[train_index, factor_i])
        y_pred[test_index] = m.predict(X[test_index, :])
    display(Markdown("##### Factor {} ".format(sfa['factor_name'][factor_i].item())))
    display(mse_performance(y[:, factor_i], y_pred))
    with plot.subplots(1, 1) as (fig, ax):
        ax.plot(y[:, factor_i], y_pred, '.')  

## Logistic Regression ##

In [ ]:
sel_patients = list(
    set(sfa['patient'].values) &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values

# Make factors binary
y_real = sfa['factors'].reindex(patient=sel_patients).transpose('patient', 'factor').values
y = np.zeros(y_real.shape, dtype=np.bool)
for factor_i in range(y.shape[1]):
    kmeans = sklearn.cluster.KMeans(2)
    kmeans.fit(y_real[:, [factor_i]])
    y[:, factor_i] = np.array(kmeans.labels_ == 1)

for factor_i in range(y.shape[1]):
    loocv = sklearn.model_selection.LeaveOneOut()
    y_pred = np.full(y.shape[0], np.nan)
    for train_index, test_index in loocv.split(X):
        m = PcaLog(10)
        m.train(X[train_index, :], y[train_index, factor_i])
        y_pred[test_index] = m.predict(X[test_index, :])
    display(Markdown("##### Factor {} ".format(sfa['factor_name'][factor_i].item())))
    display(bin_performance(y[:, factor_i], y_pred))
    plot_roc(y[:, factor_i], y_pred)

# Predict Factors from MRI features in ER+ #

## Linear Regression ##

In [ ]:
sel_patients = list(
    set(sfa['patient'].values) &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0} &
    {p for t, p in zip(clin['ihc_subtype'], clin['patient'].values)
     if t in ['ER+/HER2-']})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values
y = sfa['factors'].reindex(patient=sel_patients).transpose('patient', 'factor').values
y = (y - np.mean(y, 0, keepdims=True)) / np.std(y, 0, keepdims=True)

display(Markdown("Number of samples: {}".format(X.shape[0])))

for factor_i in range(y.shape[1]):
    loocv = sklearn.model_selection.LeaveOneOut()
    y_pred = np.full(y.shape[0], np.nan)
    for train_index, test_index in loocv.split(X):
        m = PcaLin(10)
        m.train(X[train_index, :], y[train_index, factor_i])
        y_pred[test_index] = m.predict(X[test_index, :])
    display(Markdown("##### Factor {} ".format(sfa['factor_name'][factor_i].item())))
    display(mse_performance(y[:, factor_i], y_pred))
    with plot.subplots(1, 1) as (fig, ax):
        ax.plot(y[:, factor_i], y_pred, '.')  

## Logistic Regression ##

In [ ]:
sel_patients = list(
    set(sfa['patient'].values) &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0} &
    {p for t, p in zip(clin['ihc_subtype'], clin['patient'].values)
     if t in ['ER+/HER2-']})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values

# Make factors binary
y_real = sfa['factors'].reindex(patient=sel_patients).transpose('patient', 'factor').values
y = np.zeros(y_real.shape, dtype=np.bool)
for factor_i in range(y.shape[1]):
    kmeans = sklearn.cluster.KMeans(2)
    kmeans.fit(y_real[:, [factor_i]])
    y[:, factor_i] = np.array(kmeans.labels_ == 1)

display(Markdown("Number of samples: {}".format(X.shape[0])))

for factor_i in range(y.shape[1]):
    loocv = sklearn.model_selection.LeaveOneOut()
    y_pred = np.full(y.shape[0], np.nan)
    for train_index, test_index in loocv.split(X):
        m = PcaLog(10)
        m.train(X[train_index, :], y[train_index, factor_i])
        y_pred[test_index] = m.predict(X[test_index, :])
    display(Markdown("##### Factor {} ".format(sfa['factor_name'][factor_i].item())))
    display(bin_performance(y[:, factor_i], y_pred))
    plot_roc(y[:, factor_i], y_pred)

# Predict Factors from MRI features in TN / HER2+ #

## Linear Regression ##

In [ ]:
sel_patients = list(
    set(sfa['patient'].values) &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0} &
    {p for t, p in zip(clin['ihc_subtype'], clin['patient'].values)
     if t in ['TN', 'HER2+']})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values
y = sfa['factors'].reindex(patient=sel_patients).transpose('patient', 'factor').values
y = (y - np.mean(y, 0, keepdims=True)) / np.std(y, 0, keepdims=True)

display(Markdown("Number of samples: {}".format(X.shape[0])))

for factor_i in range(y.shape[1]):
    loocv = sklearn.model_selection.LeaveOneOut()
    y_pred = np.full(y.shape[0], np.nan)
    for train_index, test_index in loocv.split(X):
        m = PcaLin(10)
        m.train(X[train_index, :], y[train_index, factor_i])
        y_pred[test_index] = m.predict(X[test_index, :])
    display(Markdown("##### Factor {} ".format(sfa['factor_name'][factor_i].item())))
    display(mse_performance(y[:, factor_i], y_pred))
    with plot.subplots(1, 1) as (fig, ax):
        ax.plot(y[:, factor_i], y_pred, '.')  

## Logistic Regression ##

In [ ]:
sel_patients = list(
    set(sfa['patient'].values) &
    {p for n, p in zip(np.isnan(mri_array).sum('variable'), mri_array['patient'].values)
     if n == 0} &
    {p for t, p in zip(clin['ihc_subtype'], clin['patient'].values)
     if t in ['TN', 'HER2+']})
X = mri_array.reindex(patient=sel_patients).transpose('patient', 'variable').values

# Make factors binary
y_real = sfa['factors'].reindex(patient=sel_patients).transpose('patient', 'factor').values
y = np.zeros(y_real.shape, dtype=np.bool)
for factor_i in range(y.shape[1]):
    kmeans = sklearn.cluster.KMeans(2)
    kmeans.fit(y_real[:, [factor_i]])
    y[:, factor_i] = np.array(kmeans.labels_ == 1)

display(Markdown("Number of samples: {}".format(X.shape[0])))

for factor_i in range(y.shape[1]):
    loocv = sklearn.model_selection.LeaveOneOut()
    y_pred = np.full(y.shape[0], np.nan)
    for train_index, test_index in loocv.split(X):
        m = PcaLog(10)
        m.train(X[train_index, :], y[train_index, factor_i])
        y_pred[test_index] = m.predict(X[test_index, :])
    display(Markdown("##### Factor {} ".format(sfa['factor_name'][factor_i].item())))
    display(bin_performance(y[:, factor_i], y_pred))
    plot_roc(y[:, factor_i], y_pred)